# Detecting drift in your images using the mobilenet Pipeline 

The following tutorial shows you 

1. How to deploy a wallaroo pipeline using faster rcnn mobilenet model created in step 2
2. Create a post-process-anomoly-detection.py file and add logic for detecting anomolies
3. We will add the post-process-anomoly-detection.py as a step in the pipeline
2. Then run inference on a sample image
3. Draw the detected objects, their bounding boxes, their classifications, and the confidence of the classifications on the provided image.
4. Note how many anomolies were detected



In [1]:
import torch
import pickle
import wallaroo
import os
import numpy as np
import json
import requests
import time
import pandas as pd
from CVDemoUtils import CVDemo

from wallaroo.assay_config import BinMode, Aggregation, Metric
from wallaroo.object import EntityNotFoundError
import matplotlib.pyplot as plt


/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wl = wallaroo.Client()

In [3]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision':
        wl.set_current_workspace(w)

In [4]:
model_name = 'mobilenet'
mobilenet_model = wl.upload_model('mobilenet', "models/mobilenet.pt.onnx")

Next we will add our post processing anomoly detection file called post-process-anomoly-detection.py.  <b>Predictions that are lower than 75% we will consider anomalies and need to be inspected.</b>

In [5]:
module_anomoly_detection = wl.upload_model("post-process-anomoly-detection", "./post-process-anomoly-detection.py").configure('python')

In [6]:
module_drift_detection = wl.upload_model("post-process-drift-detection", "./post-process-drift-detection.py").configure('python')

In [7]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("8Gi").build()

In [8]:

deployment_config = wallaroo.DeploymentConfigBuilder().image("proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:cobra-ex-input-a").memory("8Gi").build()
                                

In [9]:
#deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("8Gi").build()

In [10]:
pipeline_name = 'drift-pp-v2'
pipeline = wl.build_pipeline(pipeline_name) \
            .add_model_step(mobilenet_model) \
            .add_model_step(module_drift_detection)

pipeline.deploy(deployment_config = deployment_config)

Waiting for deployment - this will take up to 45s ....... ok


name,drift-pp-v2
created,2022-12-01 17:12:53.605165+00:00
last_updated,2022-12-01 19:36:21.829800+00:00
deployed,True
tags,
steps,post-process-drift-detection


In [11]:
#pipeline.undeploy()

## Test the pipeline by running inference on a sample image

### Prepare input image

Next we will load a sample image and resize it to the width and height required for the object detector.

We will convert the image to a numpy ndim array and add it do a dictionary

In [12]:
# The size the image will be resized to
width = 640
height = 480

imagePath = 'images/input/example/example_01.jpg'

cvDemo = CVDemo()

# The image width and height needs to be set to what the model was trained for.  In this case 640x480.
tensor, resizedImage = cvDemo.loadImageAndResize(imagePath, width, height)

# get npArray from the tensorFloat
npArray = tensor.cpu().numpy()

#creates a dictionary with the wallaroo "tensor" key and the numpy ndim array representing image as the value.
dictData = {"tensor": npArray.tolist()}


# Run inference using the SDK 

Now lets have the model detect the objects on the image by running inference and extracting the results 

In [13]:

startTime = time.time()
infResults = pipeline.infer(dictData)
endTime = time.time()

#results = infResults[0].raw
#results['original_data'] = None  # We are removing the input image json.  Not needed
#results

infResults[0].data()[0]


array([0.22962182])

In [15]:
for cnt in range(0,150):
    cvDemo.print("infer "+str(cnt))
    infResults = pipeline.infer(dictData)
    #time.sleep(1) # needed to allow the pipeline to settle in.

12-01-2022 19:37:18.539985 infer 0
12-01-2022 19:37:19.770798 infer 1
12-01-2022 19:37:20.900545 infer 2
12-01-2022 19:37:22.063543 infer 3
12-01-2022 19:37:23.257384 infer 4
12-01-2022 19:37:24.432842 infer 5
12-01-2022 19:37:25.569478 infer 6
12-01-2022 19:37:26.735554 infer 7
12-01-2022 19:37:27.869653 infer 8
12-01-2022 19:37:29.017579 infer 9
12-01-2022 19:37:30.194773 infer 10
12-01-2022 19:37:31.253381 infer 11
12-01-2022 19:37:32.349817 infer 12
12-01-2022 19:37:33.437661 infer 13
12-01-2022 19:37:34.472790 infer 14
12-01-2022 19:37:35.528370 infer 15
12-01-2022 19:37:36.586553 infer 16
12-01-2022 19:37:37.663212 infer 17
12-01-2022 19:37:38.804055 infer 18
12-01-2022 19:37:39.863845 infer 19
12-01-2022 19:37:40.979589 infer 20
12-01-2022 19:37:42.071273 infer 21
12-01-2022 19:37:43.141355 infer 22
12-01-2022 19:37:44.267374 infer 23
12-01-2022 19:37:45.298041 infer 24
12-01-2022 19:37:46.400106 infer 25
12-01-2022 19:37:47.366397 infer 26
12-01-2022 19:37:48.249410 infer 27
12

KeyboardInterrupt: 

# Extract our Anomalies

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
pd.options.display.float_format = '{:,.2f}'.format


In [16]:
import datetime
#11-16-2022 18:57:45.091063 infer 0
#11-16-2022 19:02:27.814697 infer 0
#11-16-2022 19:02:27.814697 infer 0
#11-16-2022 19:04:04.525092 infer 112

baseline_start = datetime.datetime.fromisoformat('2022-12-01T19:37:00+00:00')
baseline_end = datetime.datetime.fromisoformat('2022-12-01T19:38:039+00:00')
last_day = datetime.datetime.fromisoformat('2022-12-02T00:00:00+00:00')

Lets build our table of anomalies

In [45]:
assay_name = "example assay"
assay_builder = wl.build_assay(assay_name, pipeline, model_name, baseline_start, baseline_end)

In [46]:
assay_builder.add_iopath("output 0 0")


In [47]:

assayConfig = assay_builder.build()
assayConfig.to_json()


'{\n    "name": "example assay",\n    "pipeline_id": 3011,\n    "pipeline_name": "drift-pp-v2",\n    "active": true,\n    "status": "created",\n    "iopath": "output 0 0",\n    "baseline": {\n        "Fixed": {\n            "pipeline": "drift-pp-v2",\n            "model": "mobilenet",\n            "start_at": "2022-12-01T19:37:00+00:00",\n            "end_at": "2022-12-01T19:38:03+00:00"\n        }\n    },\n    "window": {\n        "pipeline": "drift-pp-v2",\n        "model": "mobilenet",\n        "width": "24 hours",\n        "start": null,\n        "interval": null\n    },\n    "summarizer": {\n        "type": "UnivariateContinuous",\n        "bin_mode": "Quantile",\n        "aggregation": "Density",\n        "metric": "PSI",\n        "num_bins": 5,\n        "bin_weights": null,\n        "bin_width": null,\n        "provided_edges": null,\n        "add_outlier_edges": true\n    },\n    "warning_threshold": null,\n    "alert_threshold": 0.25,\n    "run_until": null,\n    "workspace_id

In [48]:
baseline_run = assayConfig.interactive_baseline_run()
baseline_run

In [49]:
baseline_run.baseline_stats()

AttributeError: 'NoneType' object has no attribute 'baseline_stats'

In [ ]:
assay_builder.baseline_histogram()

In [ ]:
assay_builder.baseline_histogram()

In [ ]:
anomolyDf = pd.DataFrame(columns=['classification','confidence','x','y','width','height'])
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{:.2%}'.format

anomolyDf[['x', 'y','width','height']] = pd.DataFrame(anomolyBoxes)

#classes = outputs[1]['Int64']['data']
#confidences = outputs[2]['Float']['data']

idx = 0 
for idx in range(0,len(anomolyClasses)):
    anomolyDf['classification'][idx] = cvDemo.CLASSES[anomolyClasses[idx]] # Classes contains the 80 different COCO classificaitons
    anomolyDf['confidence'][idx] = anomolyConfidences[idx]
anomolyDf

In [ ]:
# Reload the sample image
tensor, resizedImage = cvDemo.loadImageAndResize(imagePath, width, height)

amber = (0,191,255)
red = (0,0,255)
blue = (255,0,0)
black = (0,0,0)

results = {
    'model_name' : model_name,
    'pipeline_name' : pipeline_name,
    'width': width,
    'height': height,    'image' : resizedImage,
    'classes' : anomolyClasses,
    'confidences' : anomolyConfidences,
    'anomaly-count' : len(anomolyConfidences),
    'boxes' : anomolyBoxes,
    'confidence-target' : 0.0,
    'inference-time': (endTime-startTime),
    'onnx-time' : int(results['elapsed']) / 1e+9,                
    'color': CVDemo.AMBER
}

image = cvDemo.drawAndDisplayDetectedObjectsWithClassification(results)

In [ ]:

pipeline.undeploy()
for d in wl.list_deployments():
    d.undeploy()
    